# reaction-network (Demo Notebook): Networks

## Author: Matthew McDermott
## Last Updated: 11/16/21

**If you use this code in your work, please consider citing the following paper:**

McDermott, M. J., Dwaraknath, S. S., and Persson, K. A. (2021). A graph-based network for predicting chemical reaction pathways in solid-state materials synthesis. 
Nature Communications, 12(1). https://doi.org/10.1038/s41467-021-23339-x

In [1]:
import logging 

from pymatgen.ext.matproj import MPRester
from rxn_network.enumerators.basic import BasicEnumerator, BasicOpenEnumerator
from rxn_network.enumerators.minimize import MinimizeGibbsEnumerator, MinimizeGrandPotentialEnumerator
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
from rxn_network.thermo.chempot_diagram import ChemicalPotentialDiagram

from rxn_network.costs.softplus import Softplus
from pymatgen.core.composition import Composition, Element
from rxn_network.entries.entry_set import GibbsEntrySet
from rxn_network.network.network import ReactionNetwork
from rxn_network.entries.nist import NISTReferenceEntry
from rxn_network.reactions.computed import ComputedReaction
from rxn_network.reactions.reaction_set import ReactionSet
from rxn_network.reactions.open import OpenComputedReaction
from rxn_network.network.entry import NetworkEntry, NetworkEntryType
from rxn_network.network.visualize import plot_network_on_graphistry, plot_network
from rxn_network.pathways.solver import PathwaySolver

import graphistry
import graph_tool.all as gt

%load_ext autoreload
%autoreload 2

In [2]:
with MPRester() as mpr:  # Insert your Materials Project API key here (if needed)
    entries = mpr.get_entries_in_chemsys("Y-Mn-O-Li-Cl-C", inc_structure="final")

  0%|          | 0/848 [00:00<?, ?it/s]

In [17]:
entry_set = GibbsEntrySet.from_entries(entries, 900, include_barin_data=False).filter_by_stability(0.025)

In [18]:
be = BasicEnumerator()
cf = Softplus(900)

In [19]:
rn = ReactionNetwork(entry_set, [BasicEnumerator()], cf)

In [20]:
rn.build()

BasicEnumerator:   0%|          | 0/57 [00:00<?, ?it/s]

In [21]:
rn.set_precursors([entry_set.get_min_entry_by_formula("Li2CO3"), 
                   entry_set.get_min_entry_by_formula("Mn2O3"), 
                   entry_set.get_min_entry_by_formula("YCl3")])

In [22]:
paths = rn.find_pathways(["YMnO3","LiCl","CO2"], k=5)

PATHS to YMnO3 

--------------------------------------- 

0.5 Mn2O3 + 1.5 Li2CO3 -> Li3MnO3 + 1.5 CO2 (dG = 0.185 eV/atom) 
Li3MnO3 + YCl3 -> YMnO3 + 3 LiCl (dG = -0.218 eV/atom) 
Total Cost: 0.529 

Mn2O3 + 2 Li2CO3 -> Li2Mn(CO3)2 + Li2MnO3 (dG = 0.042 eV/atom) 
3.429 Li2MnO3 + 3.286 Mn2O3 -> Li3MnO3 + 0.4286 Li9Mn21O40 (dG = 0.029 eV/atom) 
Li3MnO3 + YCl3 -> YMnO3 + 3 LiCl (dG = -0.218 eV/atom) 
Total Cost: 0.765 

Mn2O3 + 2 Li2CO3 -> Li2Mn(CO3)2 + Li2MnO3 (dG = 0.042 eV/atom) 
3 Li2MnO3 + 2 Mn2O3 -> Li3MnO3 + 3 LiMn2O4 (dG = 0.035 eV/atom) 
Li3MnO3 + YCl3 -> YMnO3 + 3 LiCl (dG = -0.218 eV/atom) 
Total Cost: 0.766 

Mn2O3 + Li2CO3 -> MnCO3 + Li2MnO3 (dG = 0.053 eV/atom) 
3.429 Li2MnO3 + 3.286 Mn2O3 -> Li3MnO3 + 0.4286 Li9Mn21O40 (dG = 0.029 eV/atom) 
Li3MnO3 + YCl3 -> YMnO3 + 3 LiCl (dG = -0.218 eV/atom) 
Total Cost: 0.767 

Mn2O3 + Li2CO3 -> MnCO3 + Li2MnO3 (dG = 0.053 eV/atom) 
3 Li2MnO3 + 2 Mn2O3 -> Li3MnO3 + 3 LiMn2O4 (dG = 0.035 eV/atom) 
Li3MnO3 + YCl3 -> YMnO3 + 3 LiCl (dG = 

In [23]:
ps = PathwaySolver(rn.entries, paths, Softplus(900)) # open_elem="O", chempot=0

In [24]:
product_entries = []
for i in ["YMnO3","LiCl","CO2"]:
    product_entries.append(entry_set.get_min_entry_by_formula(i))
    
net_rxn = ComputedReaction.balance(rn.precursors,product_entries)
net_rxn

YCl3 + 0.5 Mn2O3 + 1.5 Li2CO3 -> YMnO3 + 3 LiCl + 1.5 CO2

In [25]:
a = ps.solve(net_rxn, max_num_combos=5, intermediate_rxn_energy_cutoff=0.0, filter_interdependent=True)

BasicEnumerator:   0%|          | 0/30 [00:00<?, ?it/s]

PathwaySolver:   0%|          | 0/1 [00:00<?, ?it/s]

PathwaySolver:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
for path in a:
    print(path)
    print("\n")

1.5 Li2CO3 + 0.5 Mn2O3 -> 1.5 CO2 + Li3MnO3 (dG = 0.185 eV/atom) 
Li3MnO3 + YCl3 -> 3 LiCl + YMnO3 (dG = -0.218 eV/atom) 
Average Cost: 0.265


1.5 Li2CO3 + 0.5 Mn2O3 -> 1.5 CO2 + Li3MnO3 (dG = 0.185 eV/atom) 
Li3MnO3 + YCl3 -> 3 LiCl + YMnO3 (dG = -0.218 eV/atom) 
Li2CO3 -> CO2 + Li2O (dG = 0.277 eV/atom) 
Li2O + 0.5 YCl3 -> 1.5 LiCl + 0.5 LiYO2 (dG = -0.251 eV/atom) 
LiYO2 + 0.6667 Mn2O3 -> 0.3333 Li3MnO3 + YMnO3 (dG = -0.051 eV/atom) 
Average Cost: 0.266


Li2CO3 -> CO2 + Li2O (dG = 0.277 eV/atom) 
Li2O + 0.5 YCl3 -> 1.5 LiCl + 0.5 LiYO2 (dG = -0.251 eV/atom) 
LiYO2 + Mn2O3 -> LiMnO2 + YMnO3 (dG = -0.072 eV/atom) 
LiMnO2 + YClO -> LiCl + YMnO3 (dG = -0.066 eV/atom) 
LiYO2 + YCl3 -> LiCl + 2 YClO (dG = -0.164 eV/atom) 
Average Cost: 0.266


Li2CO3 -> CO2 + Li2O (dG = 0.277 eV/atom) 
Li2O + 0.5 YCl3 -> 1.5 LiCl + 0.5 LiYO2 (dG = -0.251 eV/atom) 
LiYO2 + YClO -> LiCl + Y2O3 (dG = -0.089 eV/atom) 
Mn2O3 + Y2O3 -> 2 YMnO3 (dG = -0.048 eV/atom) 
LiYO2 + YCl3 -> LiCl + 2 YClO (dG = -0.164 